In [9]:
import pandas as pd
import xlwings as xw
from copy import copy
from datetime import date
from pathlib import Path
from openpyxl import load_workbook
from openpyxl.worksheet.table import Table
from openpyxl.formula.translate import Translator
from openpyxl.utils import range_boundaries, get_column_letter

workbook = Path("test_workbook.xlsx")
csv_file = Path("Activities.csv")
assert workbook.is_file()

In [11]:
df = pd.read_csv(csv_file)
# cols = ['Aktivitetstyp', 'Datum', 'Namn', 'Distans', 'Tid']
xw.view(df)

In [24]:
def make_ref(min_col, min_row, max_col, max_row):
    return f"{get_column_letter(min_col)}{min_row}:{get_column_letter(max_col)}{max_row}"    

def append_table_values(ws, table, row_values: dict):
    # 1. Raise early if tabel has totals row since function will not work!
    if table.totalsRowShown:
        raise RuntimeError(
            "append_table_values() cannot be used on tables with a totals row; "
            "this requires row insertion logic."
        )

    # 2. Construct initial required metadata about table and new row
    columns = { tc.name: i for i, tc in enumerate(table.tableColumns) }
    min_col, min_row, max_col, max_row = range_boundaries(table.ref)
    row_i = max_row + 1
    column_index_range = range(min_col, max_col+1)

    # 3. Error checks. Check row_values type and content. Also check new row is empty!
    if not isinstance(row_values, dict):
        raise TypeError(f"row_values has to be a dict not a {type(row_values)}!")

    faulty_keys = row_values.keys() - columns.keys()
    if faulty_keys:
        bad = ", ".join(sorted(map(str, faulty_keys)))
        raise ValueError(f"row_values contains invalid column names: {bad}")

    if any(ws.cell(row=row_i, column=c).value is not None for c in column_index_range):
        raise ValueError(f"Row {row_i} is not empty in the table area; would overwrite data!")

    # 4. Set cell values for new row
    for col, val in row_values.items():
        col_i = min_col + columns[col]
        ws.cell(row=row_i, column=col_i, value=val)

    # 5. Fix formating of new row and its cells. Also expand formulas to new rows!
    if max_row > min_row:
        src_row = ws.row_dimensions[max_row]
        dst_row = ws.row_dimensions[row_i]
        dst_row.height = src_row.height

        for col_i in column_index_range:
            src = ws.cell(row=max_row, column=col_i)
            dst = ws.cell(row=row_i, column=col_i)

            if src.has_style:
                dst._style = copy(src._style)
            if dst.value is None and (isinstance(src.value, str) and src.value.startswith("=")):
                dst.value = Translator(src.value, origin=src.coordinate).translate_formula(dst.coordinate)

    # 6. Resize table to include new row
    new_ref = make_ref(min_col, min_row, max_col, row_i)
    table.ref = new_ref

In [25]:
wb = load_workbook(workbook)
ws = wb["Running sessions"]
tbl: Table = ws.tables["tblRun"]

test_date = date(2025, 12, 24)
row_dct = {"Date": test_date, "Name": "Test append_table_values", "Distance":9500, "Load": 196}
append_table_values(ws, tbl, row_dct)
wb.save(workbook)
wb.close()